In [1]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("churn-data-3333.csv")

In [2]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()
num_df.head()

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,128,415,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


In [3]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)
cat_df=cat_df.drop(['phone number'], axis=1)



#BOX-COX
#lemda=0.5
#num_df=(num_df**lemda)
#num_df=num_df-1
#num_df[num_df < 0]=0
#num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
#num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
num_df=round(np.log(num_df.add(1)),2)
num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log

num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn'] 
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [4]:
X.shape

(3333, 19)

In [5]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
17,international_plan,203.244178
2,number_vmail_messages,80.494379
18,voice_mail_plan,25.156959
15,number_customer_service_calls,23.087955
5,total_day_charge,2.490080
16,state,1.701057
3,total_day_minutes,1.679470
13,total_intl_calls,1.640827
8,total_eve_charge,0.607927
12,total_intl_minutes,0.563038


In [6]:
y_train=y
x_train_chi = select_feature.transform(X)
x_train_chi.shape

(3333, 15)

In [7]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

2648 202 318 165
pod:  0.3416149068322981
pof:  0.07087719298245614
AUC:  0.635368856924921


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min


{'var_smoothing': 0.0006579332246575676}
GaussianNB(priors=None, var_smoothing=0.0006579332246575676)
0.8591859185918592


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  1.1min finished


26710 23728 21590 27972
pod:  0.5643840038739357
pof:  0.4704389547563345
AUC:  0.5469725245588006


In [10]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=0.0006579332246575676)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

2825 25 442 41
pod:  0.08488612836438923
pof:  0.008771929824561403
AUC:  0.5380570992699139
accuracy:  0.8598859885988599


In [1]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 234 tasks      | elapsed:  1.9min


{'C': 6, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'lbfgs'}
LogisticRegression(C=6, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
0.8610861086108611


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  2.0min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [14]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=6, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

2799 51 413 70
pod:  0.14492753623188406
pof:  0.017894736842105262
AUC:  0.5635163996948894
accuracy:  0.8607860786078608


In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'uniform'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
0.8631863186318632


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    0.9s finished


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear'], 
                     'gamma': [0.001, 0.01, 0.1, 1, 1e-3, 1e-4], 
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} ]

grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 410.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 1993.6min


In [18]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform' )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


2818 32 431 52
pod:  0.10766045548654245
pof:  0.011228070175438596
AUC:  0.5482161926555519
accuracy:  0.8610861086108611


In [19]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


2827 23 456 27
pod:  0.055900621118012424
pof:  0.008070175438596491
AUC:  0.5239152228397079
accuracy:  0.8562856285628563


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  2.9min finished


{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'uniform'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
0.8631863186318632


In [21]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

{'bootstrap': True, 'max_depth': 110, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 1000}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=110, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.9510951095109511


In [22]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 110, max_features= 3, min_samples_leaf= 3, min_samples_split= 8, n_estimators= 1000)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2831 19 134 349
pod:  0.722567287784679
pof:  0.006666666666666667
AUC:  0.8579503105590062
accuracy:  0.9540954095409541


In [23]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2849 1 473 10
pod:  0.020703933747412008
pof:  0.0003508771929824561
AUC:  0.5101765282772147


In [24]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2724 126 130 353
pod:  0.7308488612836439
pof:  0.04421052631578947
AUC:  0.8433191674839272
accuracy:  0.9231923192319232


In [25]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2705 145 129 354
pod:  0.7329192546583851
pof:  0.05087719298245614
AUC:  0.8410210308379645


In [26]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2817 33 126 357
pod:  0.7391304347826086
pof:  0.011578947368421053
AUC:  0.8637757437070938
accuracy:  0.9522952295229523


In [9]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

27860 22578 18243 31319
pod:  0.631915580485049
pof:  0.44763868511836313
AUC:  0.592138447683343
accuracy:  0.59179


In [27]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 0s 191us/step - loss: 0.4660 - acc: 0.8458 - val_loss: 0.4316 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 55us/step - loss: 0.4072 - acc: 0.8587 - val_loss: 0.4288 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 78us/step - loss: 0.4102 - acc: 0.8587 - val_loss: 0.4288 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 75us/step - loss: 0.3997 - acc: 0.8587 - val_loss: 0.4171 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 50us/step - loss: 0.3904 - acc: 0.8587 - val_loss: 0.4182 - val_acc: 0.8417
Epoch 6/30
2399/2399 [==============================] - 0s 51us/step - loss: 0.3859 - acc: 0.8587 - val_loss: 0.3993 - val_acc: 0.8417
Epoch 7/30
2399/2399 [

 [1.]]
pod 1st:  0.10204081632653061
Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 0s 205us/step - loss: 0.4481 - acc: 0.8504 - val_loss: 0.4334 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 50us/step - loss: 0.4192 - acc: 0.8587 - val_loss: 0.4556 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 51us/step - loss: 0.4130 - acc: 0.8587 - val_loss: 0.4366 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 49us/step - loss: 0.4009 - acc: 0.8587 - val_loss: 0.4298 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 50us/step - loss: 0.4065 - acc: 0.8587 - val_loss: 0.4504 - val_acc: 0.8417
Epoch 6/30
2399/2399 [==============================] - 0s 50us/step - loss: 0.4002 - acc: 0.8587 - val_loss: 0.4251 - val_acc: 0.8417
Epoch 7/30
2399/2399 [==============================] - 0s 56us/step - loss: 0.3981 - acc: 0.8587 - val_l

pod 1st:  0.12244897959183673
Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 1s 224us/step - loss: 0.4494 - acc: 0.8558 - val_loss: 0.4367 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 52us/step - loss: 0.4079 - acc: 0.8587 - val_loss: 0.4487 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 52us/step - loss: 0.4082 - acc: 0.8587 - val_loss: 0.4367 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 58us/step - loss: 0.3946 - acc: 0.8587 - val_loss: 0.4228 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 49us/step - loss: 0.3943 - acc: 0.8587 - val_loss: 0.4379 - val_acc: 0.8417
Epoch 6/30
2399/2399 [==============================] - 0s 50us/step - loss: 0.3891 - acc: 0.8587 - val_loss: 0.4128 - val_acc: 0.8417
Epoch 7/30
2399/2399 [==============================] - 0s 50us/step - loss: 0.3876 - acc: 0.8587 - val_loss: 0.

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 294us/step - loss: 0.4277 - acc: 0.8483 - val_loss: 0.4335 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 53us/step - loss: 0.4037 - acc: 0.8583 - val_loss: 0.4142 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 53us/step - loss: 0.3946 - acc: 0.8583 - val_loss: 0.4112 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 53us/step - loss: 0.3827 - acc: 0.8583 - val_loss: 0.4181 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 51us/step - loss: 0.3865 - acc: 0.8583 - val_loss: 0.4265 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.3874 - acc: 0.8583 - val_loss: 0.4060 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 52us/step - loss: 0.3736 - acc: 0.8583 - val_loss: 0.4007 - val_acc: 0.8417
Epoch 8

...

pod 1st:  0.16666666666666666
Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 305us/step - loss: 0.4495 - acc: 0.8500 - val_loss: 0.4271 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 55us/step - loss: 0.4078 - acc: 0.8583 - val_loss: 0.4167 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 54us/step - loss: 0.3957 - acc: 0.8583 - val_loss: 0.4133 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 53us/step - loss: 0.3920 - acc: 0.8583 - val_loss: 0.4019 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 55us/step - loss: 0.3871 - acc: 0.8583 - val_loss: 0.4143 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 54us/step - loss: 0.3782 - acc: 0.8579 - val_loss: 0.3944 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 54us/step - loss: 0.3845 - acc: 0.8588 - val_loss: 0.

pod 1st:  0.0625
Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 331us/step - loss: 0.4461 - acc: 0.8467 - val_loss: 0.4401 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 59us/step - loss: 0.4110 - acc: 0.8583 - val_loss: 0.4325 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.4039 - acc: 0.8583 - val_loss: 0.4248 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.3965 - acc: 0.8583 - val_loss: 0.4168 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.3981 - acc: 0.8583 - val_loss: 0.4145 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.3912 - acc: 0.8583 - val_loss: 0.4297 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 71us/step - loss: 0.3873 - acc: 0.8583 - val_loss: 0.4020 - val_ac

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 387us/step - loss: 0.4359 - acc: 0.8521 - val_loss: 0.4459 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.4150 - acc: 0.8583 - val_loss: 0.4498 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 59us/step - loss: 0.4113 - acc: 0.8583 - val_loss: 0.4541 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.4110 - acc: 0.8583 - val_loss: 0.4455 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 59us/step - loss: 0.4071 - acc: 0.8583 - val_loss: 0.4279 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 65us/step - loss: 0.4030 - acc: 0.8583 - val_loss: 0.4216 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.4034 - acc: 0.8583 - val_loss: 0.4342 - val_acc: 0.8417
Epoch 8

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 448us/step - loss: 0.4401 - acc: 0.8563 - val_loss: 0.5185 - val_acc: 0.8233
Epoch 2/30
2400/2400 [==============================] - 0s 75us/step - loss: 0.4071 - acc: 0.8629 - val_loss: 0.4878 - val_acc: 0.8233
Epoch 3/30
2400/2400 [==============================] - 0s 80us/step - loss: 0.4000 - acc: 0.8629 - val_loss: 0.4775 - val_acc: 0.8233
Epoch 4/30
2400/2400 [==============================] - 0s 62us/step - loss: 0.3963 - acc: 0.8629 - val_loss: 0.4624 - val_acc: 0.8233
Epoch 5/30
2400/2400 [==============================] - 0s 60us/step - loss: 0.3857 - acc: 0.8629 - val_loss: 0.4816 - val_acc: 0.8233
Epoch 6/30
2400/2400 [==============================] - 0s 71us/step - loss: 0.3794 - acc: 0.8629 - val_loss: 0.4271 - val_acc: 0.8233
Epoch 7/30
2400/2400 [==============================] - 0s 68us/step - loss: 0.3746 - acc: 0.8629 - val_loss: 0.4243 - val_acc: 0.8233
Epoch 8

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 432us/step - loss: 0.4471 - acc: 0.8504 - val_loss: 0.4744 - val_acc: 0.8233
Epoch 2/30
2400/2400 [==============================] - 0s 64us/step - loss: 0.4117 - acc: 0.8629 - val_loss: 0.4650 - val_acc: 0.8233
Epoch 3/30
2400/2400 [==============================] - 0s 62us/step - loss: 0.4021 - acc: 0.8629 - val_loss: 0.4597 - val_acc: 0.8233
Epoch 4/30
2400/2400 [==============================] - 0s 60us/step - loss: 0.4006 - acc: 0.8629 - val_loss: 0.4599 - val_acc: 0.8233
Epoch 5/30
2400/2400 [==============================] - 0s 60us/step - loss: 0.3960 - acc: 0.8629 - val_loss: 0.4897 - val_acc: 0.8233
Epoch 6/30
2400/2400 [==============================] - 0s 60us/step - loss: 0.3891 - acc: 0.8629 - val_loss: 0.4452 - val_acc: 0.8233
Epoch 7/30
2400/2400 [==============================] - 0s 65us/step - loss: 0.3866 - acc: 0.8629 - val_loss: 0.4735 - val_acc: 0.8233
Epoch 8

In [22]:
tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)

21715 28723 14408 35154


In [28]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

Train on 2399 samples, validate on 600 samples
Epoch 1/5
2399/2399 [==============================] - 2s 645us/step - loss: 0.5591 - acc: 0.8212 - val_loss: 0.4372 - val_acc: 0.8417
Epoch 2/5
2399/2399 [==============================] - 0s 85us/step - loss: 0.4075 - acc: 0.8587 - val_loss: 0.4374 - val_acc: 0.8417
Epoch 3/5
2399/2399 [==============================] - 0s 88us/step - loss: 0.4053 - acc: 0.8587 - val_loss: 0.4419 - val_acc: 0.8417
Epoch 4/5
2399/2399 [==============================] - 0s 84us/step - loss: 0.4072 - acc: 0.8587 - val_loss: 0.4355 - val_acc: 0.8417
Epoch 5/5
2399/2399 [==============================] - 0s 87us/step - loss: 0.4029 - acc: 0.8587 - val_loss: 0.4350 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 2s 841us/step - loss: 0.5251 - acc: 0.8533 - val_loss: 0.4428 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 86us/step - loss: 0.4083 - acc: 0.8583 - val_loss: 0.4385 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 85us/step - loss: 0.4081 - acc: 0.8583 - val_loss: 0.4355 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 88us/step - loss: 0.4064 - acc: 0.8583 - val_loss: 0.4344 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 85us/step - loss: 0.4031 - acc: 0.8583 - val_loss: 0.4335 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 2s 999us/step - loss: 0.5377 - acc: 0.8583 - val_loss: 0.4375 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 85us/step - loss: 0.4066 - acc: 0.8583 - val_loss: 0.4406 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 86us/step - loss: 0.4057 - acc: 0.8583 - val_loss: 0.4340 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 91us/step - loss: 0.4076 - acc: 0.8583 - val_loss: 0.4335 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 87us/step - loss: 0.4031 - acc: 0.8583 - val_loss: 0.4358 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 3s 1ms/step - loss: 0.5039 - acc: 0.8629 - val_loss: 0.4685 - val_acc: 0.8233
Epoch 2/5
2400/2400 [==============================] - 0s 88us/step - loss: 0.4004 - acc: 0.8629 - val_loss: 0.4797 - val_acc: 0.8233
Epoch 3/5
2400/2400 [==============================] - 0s 88us/step - loss: 0.4001 - acc: 0.8629 - val_loss: 0.4685 - val_acc: 0.8233
Epoch 4/5
2400/2400 [==============================] - 0s 89us/step - loss: 0.3981 - acc: 0.8629 - val_loss: 0.4976 - val_acc: 0.8233
Epoch 5/5
2400/2400 [==============================] - 0s 89us/step - loss: 0.3981 - acc: 0.8629 - val_loss: 0.4612 - val_acc: 0.8233
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0